In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

In [2]:
from phenum.symmetry import get_lattice_pointGroup

In [3]:
import numpy as np
from numpy.linalg import inv, norm
from itertools import product
from copy import deepcopy

from BZI.symmetry import (find_orbitals, make_ptvecs,
                          make_rptvecs, reduce_kpoint_list,
                          find_orbits, find_orbitals, 
                          brillouin_zone_mapper, reduce_kpoint_list)
from BZI.sampling import make_cell_points, sphere_pts
from BZI.plots import plot_mesh
from BZI.symmetry import minkowski_reduce_basis

In [4]:
from BZI.pseudopots import Al_PP

In [5]:
lat_consts = [1.5*2*np.pi]*3
lat_angles = [np.pi/2]*3
lat_center = 'body'
lat_vecs = make_ptvecs(lat_center, lat_consts, lat_angles)
rlat_vecs = make_rptvecs(lat_vecs)

HNF = [[1,0,0],[0,2,0],[1,2,3]]
grid_vecs = np.dot(rlat_vecs, inv(HNF))

grid_offset = [0.98, 0.23, 0.77]
# offset = np.dot(inv(grid_vecs), np.dot(rlat_vecs, grid_offset))
grid = np.array(make_cell_points(rlat_vecs, grid_vecs, grid_offset))

In [6]:
print("shift = (/ " + "_dp, ".join(map(str, np.round(grid_offset, 4))) + "_dp" + " /)")

# Print the reciprocal lattice vectors in a format that is 
# easy to copy and paste into Fortran code.
for i, r in enumerate(rlat_vecs):
    if i == 0:
        print("R = transpose(reshape((/ " + 
              "_dp, ".join(map(str, np.round(r, 12))) + "_dp, &")
    elif i == 1:
        print("                          " + 
              "_dp, ".join(map(str, np.round(r, 12))) + "_dp, &")
    else: 
        print("                          " + 
              "_dp, ".join(map(str, np.round(r, 12))) + "_dp /)," + 
             "(/3,3/)))")
        
# Print the reciprocal lattice vectors in a format that is 
# easy to copy and paste into Fortran code.
for i, r in enumerate(HNF):
    if i == 0:
        print("H = transpose(reshape((/ " + 
              ", ".join(map(str, np.round(r, 12))) + ", &")
    elif i == 1:
        print("                          " + 
              ", ".join(map(str, np.round(r, 12))) + ", &")
    else:
        print("                          " + 
              ", ".join(map(str, np.round(r, 12))) + " /)," + 
             "(/3,3/)))")

shift = (/ 0.98_dp, 0.23_dp, 0.77_dp /)
R = transpose(reshape((/ -0.0_dp, 0.666666666667_dp, 0.666666666667_dp, &
                          0.666666666667_dp, 0.0_dp, 0.666666666667_dp, &
                          0.666666666667_dp, 0.666666666667_dp, 0.0_dp /),(/3,3/)))
H = transpose(reshape((/ 1, 0, 0, &
                          0, 2, 0, &
                          1, 2, 3 /),(/3,3/)))


In [7]:
list(product([-1,0], repeat=3))

[(-1, -1, -1),
 (-1, -1, 0),
 (-1, 0, -1),
 (-1, 0, 0),
 (0, -1, -1),
 (0, -1, 0),
 (0, 0, -1),
 (0, 0, 0)]

In [8]:
rlat_vecs

array([[-0.        ,  0.66666667,  0.66666667],
       [ 0.66666667,  0.        ,  0.66666667],
       [ 0.66666667,  0.66666667,  0.        ]])

In [9]:
reduce_kpoint_list(grid, rlat_vecs, grid_vecs, grid_offset)

([array([ 0.64555556,  1.22222222,  0.73      ]),
  array([ 0.20111111,  0.77777778,  0.73      ]),
  array([ 0.42333333,  1.        ,  0.73      ]),
  array([ 0.75666667,  1.        ,  1.06333333]),
  array([ 0.97888889,  1.22222222,  1.06333333]),
  array([ 0.53444444,  0.77777778,  1.06333333])],
 [1, 1, 1, 1, 1, 1])

In [10]:
bzgrid, weights = brillouin_zone_mapper(grid, rlat_vecs, grid_vecs, grid_offset)

In [11]:
bzgrid

array([[-0.02111111, -0.11111111,  0.06333333],
       [ 0.20111111,  0.11111111,  0.06333333],
       [-0.24333333, -0.33333333,  0.06333333],
       [ 0.09      ,  0.33333333, -0.27      ],
       [-0.35444444, -0.11111111, -0.27      ],
       [-0.13222222,  0.11111111, -0.27      ]])

In [12]:
eps = 15
mink_lat = minkowski_reduce_basis(rlat_vecs, eps)

In [13]:
mink_lat

array([[ 0.66666667,  0.        , -0.        ],
       [ 0.        ,  0.66666667,  0.66666667],
       [ 0.66666667, -0.66666667,  0.66666667]])

In [14]:
np.dot(rlat_vecs, inv(mink_lat))

array([[  2.77555756e-17,   1.00000000e+00,   0.00000000e+00],
       [  5.00000000e-01,   5.00000000e-01,   5.00000000e-01],
       [  1.50000000e+00,   5.00000000e-01,  -5.00000000e-01]])

In [15]:
np.dot(inv(mink_lat), rlat_vecs)

array([[ 0.,  1.,  1.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]])

In [16]:
rlat_vecs

array([[-0.        ,  0.66666667,  0.66666667],
       [ 0.66666667,  0.        ,  0.66666667],
       [ 0.66666667,  0.66666667,  0.        ]])

In [17]:
grid_vecs

array([[-0.22222222,  0.11111111,  0.22222222],
       [ 0.44444444, -0.22222222,  0.22222222],
       [ 0.66666667,  0.33333333,  0.        ]])

In [18]:
grid_offset

[0.98, 0.23, 0.77]

In [30]:
tmink_lat = np.transpose(mink_lat)

In [27]:
pt = np.array([ 0.97888889,  1.22222222,  1.06333333])

In [34]:
mink_lat

array([[ 0.66666667,  0.        , -0.        ],
       [ 0.        ,  0.66666667,  0.66666667],
       [ 0.66666667, -0.66666667,  0.66666667]])

In [33]:
pt + np.dot([-2,0,0], mink_lat)

array([-0.35444444,  1.22222222,  1.06333333])

In [28]:
pt + np.dot(mink_lat, [-2,0,0])

array([-0.35444444,  1.22222222, -0.27      ])

In [29]:
pt + -1*mink_lat[:,0] + 0*mink_lat[:,1] + 0*mink_lat[:,2]

array([ 0.31222222,  1.22222222,  0.39666666])